# Working with EXIF metadata in C# with ImageSharp

## Setup

We need to reference the `SixLabors.ImageSharp` [NuGet package](https://sixlabors.com/products/imagesharp/). 

**Note:** If you use ImageSharp for profit, you might need to purchase a [commercial license subscription](https://sixlabors.com/pricing/), which can get quite expensive. If you need free solution for commercial use, use for example the `MetadataExtractor` library.

In [79]:
#r "nuget: SixLabors.ImageSharp"

Installed Packages SixLabors.ImageSharp, 3.1.11

We define the namespaces we will be using later:

In [80]:
using SixLabors.ImageSharp;
using SixLabors.ImageSharp.Metadata.Profiles.Exif;

Also define the name of image file we are working with:

In [81]:
var imageFileName = @"demo.jpg";

## Show raw data

In [82]:
using (Image image = Image.Load(imageFileName)) {
    var exifProfile = image.Metadata.ExifProfile;
    if (exifProfile != null) {
        var results = exifProfile.Values.Select(p=> new {
            Tag = p.Tag.ToString(),
            Type = p.DataType,
            IsArray = p.IsArray,
            Value = p.GetValue()
        });
        results.DisplayTable();
    } else {
        Console.WriteLine("No EXIF metadata found.");
    }
}

Tag Type IsArray Value ImageWidth Short False 4000 ImageLength Short False 1800 Orientation Short False 1 ResolutionUnit Short False 2 YCbCrPositioning Short False 2 544 Long False 0 545 Long False 0 546 Long False 0 547 Long False 0 548 Long False 0 ExposureProgram Short False 0 ISOSpeedRatings Short True [ 100 ] SensitivityType Short False 0 RecommendedExposureIndex Long False 0 ExifVersion Undefined True [ 48, 50, 50, 48 ] ComponentsConfiguration Undefined True [ 1, 2, 3, 0 ] MeteringMode Short False 2 LightSource Short False 255 Flash Short False 0 SubsecTime Ascii False 503 (43 more)

## Format the data usefully

Define helper method to convert raw values to some readable output:

In [83]:
public static string ToPrettyString(IExifValue exif) {

    static string GetLabel(object value, string labels) {
        var labelArray = labels.Split(',').Select(s => s.Trim()).ToArray();
        var index = (ushort)value;
        return index < 0 || index >= labelArray.LongLength ? labelArray[0] : labelArray[index];
    }

    static IEnumerable<string> GetFlashFlags(object value) {
        var flags = (ushort)value;
        yield return (flags & 1) != 0 ? "Fired" : "Not fired";
        if ((flags & 2) != 0) yield return "Strobe return light detected";
        if ((flags & 4) != 0) yield return "Strobe return light not detected";
        if ((flags & 8) != 0) yield return "Compulsory flash mode";
        if ((flags & 16) != 0) yield return "Auto mode";
        if ((flags & 32) != 0) yield return "No flash function";
        if ((flags & 64) != 0) yield return "Red eye reduction mode";
    }

    static string GetGpsCoordinate(object value) {
        try {
            var array = value as Rational[];
            return $"{array[0]}°{array[1]}'{System.Xml.XmlConvert.ToString(array[2].ToDouble())}\"";
        } catch (Exception) {
            return null;
        }
    }

    // Special cases
    if (exif.Tag == ExifTag.Orientation) return GetLabel(exif.GetValue(), "Unknown, Top Left, Top Right, Bottom Right, Bottom Left, Left Top, Right Top, Right Bottom, Left Bottom");
    if (exif.Tag == ExifTag.ExposureProgram) return GetLabel(exif.GetValue(), "Unknown, Manual, Auto, Aperture priority, Shutter speed priority, Creative, Action, Portrait, Landscape, Bulb");
    if (exif.Tag == ExifTag.MeteringMode) return GetLabel(exif.GetValue(), "Unknown, Average, Center Weighted Average, Spot, Multi Spot, Multi Segment, Partial");
    if (exif.Tag == ExifTag.LightSource) return GetLabel(exif.GetValue(), "Unknown, Daylight, Fluorescent, Tungsten, Flash, Sunny, Cloudy, Shade, Daylight Fluorescent, Day White Fluorescent, Cool White Fluorescent, White Fluorescent, Warm White Fluorescent, Standard Light A, Standard Light B, StandardLight C, D55, D65, D75, D50, ISO Studio Tungsten");
    if (exif.Tag == ExifTag.ExposureMode) return GetLabel(exif.GetValue(), "Auto, Manual, Auto bracket");
    if (exif.Tag == ExifTag.WhiteBalance) return GetLabel(exif.GetValue(), "Auto, Manual");
    if (exif.Tag == ExifTag.FNumber) return ((Rational)exif.GetValue()).ToDouble().ToString("N1", System.Globalization.CultureInfo.InvariantCulture);
    if (exif.Tag == ExifTag.SceneCaptureType) return GetLabel(exif.GetValue(), "Standard, Landscape, Portrait, Night, Other");
    if (exif.Tag == ExifTag.DateTimeOriginal || exif.Tag == ExifTag.DateTimeDigitized) return DateTime.TryParseExact(exif.GetValue().ToString(), @"yyyy\:MM\:dd HH\:mm\:ss", null, System.Globalization.DateTimeStyles.None, out var dt) ? System.Xml.XmlConvert.ToString(dt, System.Xml.XmlDateTimeSerializationMode.RoundtripKind) : null;
    if (exif.Tag == ExifTag.Flash) return string.Join(", ", GetFlashFlags(exif.GetValue()));
    if (exif.Tag == ExifTag.GPSDestLatitude || exif.Tag == ExifTag.GPSDestLongitude || exif.Tag == ExifTag.GPSLatitude || exif.Tag == ExifTag.GPSLongitude) return GetGpsCoordinate(exif.GetValue());
    if (exif.Tag == ExifTag.ExifVersion) return System.Text.Encoding.UTF8.GetString((byte[])exif.GetValue());
    if (exif.Tag == ExifTag.ComponentsConfiguration) return string.Join(", ", ((byte[])exif.GetValue()).Select(b => GetLabel((ushort)b, "-, Y, Cb, Cr, R, G, B")));

    // Array of generic values
    if (exif.IsArray) {
        var s = string.Empty;
        foreach (var item in exif.GetValue() as Array) {
            s += item.ToString() + " ";
        }
        return s.Trim();
    }

    // If no special case occurred, just return ToString()
    return exif.GetValue().ToString();
}

// Use the helper method to display data
using (Image image = Image.Load(imageFileName)) {
    var exifProfile = image.Metadata.ExifProfile;
    if (exifProfile != null) {
        var results = exifProfile.Values.Select(p=> new {
            Tag = p.Tag.ToString(),
            Type = p.DataType,
            IsArray = p.IsArray,
            RawValue = p.GetValue(),
            FormattedValue = ToPrettyString(p),
        });
        results.DisplayTable();
    } else {
        Console.WriteLine("No EXIF metadata found.");
    }
}

Tag Type IsArray RawValue FormattedValue ImageWidth Short False 4000 4000 ImageLength Short False 1800 1800 Orientation Short False 1 Top Left ResolutionUnit Short False 2 2 YCbCrPositioning Short False 2 2 544 Long False 0 0 545 Long False 0 0 546 Long False 0 0 547 Long False 0 0 548 Long False 0 0 ExposureProgram Short False 0 Unknown ISOSpeedRatings Short True [ 100 ] 100 SensitivityType Short False 0 0 RecommendedExposureIndex Long False 0 0 ExifVersion Undefined True [ 48, 50, 50, 48 ] 0220 ComponentsConfiguration Undefined True [ 1, 2, 3, 0 ] Y, Cb, Cr, - MeteringMode Short False 2 Center Weighted Average LightSource Short False 255 Unknown Flash Short False 0 Not fired SubsecTime Ascii False 503 503 (43 more)